In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import glob
import re

In [ ]:
p_1_files = glob.glob(f'data_2016/*_0_*.csv')
p_2_files = glob.glob(f'data_2016/*_1_*.csv')
p_3_files = glob.glob(f'data_2016/*_2_*.csv')
p_4_files = glob.glob(f'data_2016/*_3_*.csv')
p_5_files = glob.glob(f'data_2016/*_4_*.csv')
p_6_files = glob.glob(f'data_2016/*_5_*.csv')
p_7_files = glob.glob(f'data_2016/*_6_*.csv')
p_8_files = glob.glob(f'data_2016/*_7_*.csv')
p_9_files = glob.glob(f'data_2016/*_8_*.csv')
p_10_files = glob.glob(f'data_2016/*_9_*.csv')

print(len(p_1_files))
print(len(p_2_files))
print(len(p_3_files))
print(len(p_4_files))
print(len(p_5_files))
print(len(p_6_files))
print(len(p_7_files))
print(len(p_8_files))
print(len(p_9_files))
print(len(p_10_files))

In [ ]:
dfs_p1_16 = [pd.read_csv(filename) for filename in p_1_files]
dfs_p2_16 = [pd.read_csv(filename) for filename in p_2_files]
dfs_p3_16 = [pd.read_csv(filename) for filename in p_3_files]
dfs_p4_16 = [pd.read_csv(filename) for filename in p_4_files]
dfs_p5_16 = [pd.read_csv(filename) for filename in p_5_files]
dfs_p6_16 = [pd.read_csv(filename) for filename in p_6_files]
dfs_p7_16 = [pd.read_csv(filename) for filename in p_7_files]
dfs_p8_16 = [pd.read_csv(filename) for filename in p_8_files]
dfs_p9_16 = [pd.read_csv(filename) for filename in p_9_files]
dfs_p10_16 = [pd.read_csv(filename) for filename in p_10_files]

big_frame_p1_16 = pd.concat(dfs_p1_16, ignore_index=True)
big_frame_p2_16 = pd.concat(dfs_p2_16, ignore_index=True)
big_frame_p3_16 = pd.concat(dfs_p3_16, ignore_index=True)
big_frame_p4_16 = pd.concat(dfs_p4_16, ignore_index=True)
big_frame_p5_16 = pd.concat(dfs_p5_16, ignore_index=True)
big_frame_p6_16 = pd.concat(dfs_p6_16, ignore_index=True)
big_frame_p7_16 = pd.concat(dfs_p7_16, ignore_index=True)
big_frame_p8_16 = pd.concat(dfs_p8_16, ignore_index=True)
big_frame_p9_16 = pd.concat(dfs_p9_16, ignore_index=True)
big_frame_p10_16 = pd.concat(dfs_p10_16, ignore_index=True)

big_frame_list_16 = [big_frame_p1_16, big_frame_p2_16, big_frame_p3_16, big_frame_p4_16, big_frame_p5_16, 
                  big_frame_p6_16, big_frame_p7_16, big_frame_p8_16, big_frame_p9_16, big_frame_p10_16]